In [1]:
import os
import pandas as pd
from ipywidgets.widgets import Dropdown, interact
from ipywidgets import Button, HBox, VBox, interactive_output

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (10,8)

In [2]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess, Fourier
from sklearn.linear_model import Ridge

In [3]:
all_data = {}
for file in os.listdir('data'):
    filename = file.split('.')[0]
    all_data[filename] = pd.read_parquet('data/{}'.format(file),)
    
all_data.keys()

dict_keys(['holidays_events', 'oil', 'sample_submission', 'stores', 'test', 'train', 'transactions'])

In [4]:
train, stores, holidays_events, oil = (all_data['train'], all_data['stores'], 
         all_data['holidays_events'], all_data['oil'])
for data in (train, holidays_events, oil):
    data['date'] = data['date'].astype('datetime64[ns]')

In [5]:
train.drop(['id', 'onpromotion'], axis=1, inplace=True)
train.date = train.date.dt.to_period('D')
train = train.set_index(['store_nbr', 'family', 'date']).sort_index()
train

sales
store_nbr family     date             
1         AUTOMOTIVE 2013-01-01    0.0
                     2013-01-02    2.0
                     2013-01-03    3.0
                     2013-01-04    3.0
                     2013-01-05    5.0
...                                ...
54        SEAFOOD    2017-08-11    0.0
                     2017-08-12    1.0
                     2017-08-13    2.0
                     2017-08-14    0.0
                     2017-08-15    3.0

[3000888 rows x 1 columns]

In [14]:
start_date = '2013-01-01'
end_date = '2017-08-01'

In [15]:
y = train.unstack(['store_nbr', 'family']).loc[start_date:end_date]
y

sales                                                \
store_nbr          1                                                  
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2013-01-01        0.0       0.0    0.0       0.0   0.0      0.00000   
2013-01-02        2.0       0.0    2.0    1091.0   0.0    470.65200   
2013-01-03        3.0       0.0    0.0     919.0   0.0    310.65500   
2013-01-04        3.0       0.0    3.0     953.0   0.0    198.36600   
2013-01-05        5.0       0.0    3.0    1160.0   0.0    301.05700   
...               ...       ...    ...       ...   ...          ...   
2017-07-28        7.0       0.0    2.0    2358.0   0.0    403.64500   
2017-07-29        4.0       0.0    3.0    2161.0   0.0    330.03500   
2017-07-30        1.0       0.0    2.0    1212.0   0.0    153.80800   
2017-07-31        8.0       0.0    3.0    2414.0   1.0    370.99400   
2017-08-01        5.0       0.0    4.0    2627.0   0.0    373.70898   

                                                 ...                       \
store_nbr                                        ...        54              
family     CELEBRATION CLEANING  DAIRY     DELI  ... MAGAZINES      MEATS   
date                                             ...                        
2013-01-01         0.0      0.0    0.0    0.000  ...       0.0   0.000000   
2013-01-02         0.0   1060.0  579.0  164.069  ...       0.0  73.771000   
2013-01-03         0.0    836.0  453.0  151.582  ...       0.0  50.257000   
2013-01-04         0.0    827.0  460.0  131.411  ...       0.0  40.223000   
2013-01-05         0.0    811.0  464.0  118.613  ...       0.0  43.431000   
...                ...      ...    ...      ...  ...       ...        ...   
2017-07-28        20.0    714.0  711.0  178.409  ...       1.0  44.336998   
2017-07-29         6.0    667.0  676.0  122.680  ...       3.0  50.606000   
2017-07-30         0.0    238.0  316.0   54.296  ...       3.0  76.667000   
2017-07-31         3.0    678.0  727.0  189.110  ...       1.0  42.387000   
2017-08-01        11.0    867.0  668.0  133.588  ...       0.0  72.666000   

                                                                          \
store_nbr                                                                  
family     PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS    POULTRY   
date                                                                       
2013-01-01           0.0          0.0                     0.0   0.000000   
2013-01-02         228.0          0.0                     0.0  15.514000   
2013-01-03         156.0          0.0                     0.0   4.313000   
2013-01-04         146.0          0.0                     0.0  26.743000   
2013-01-05         205.0          0.0                     0.0  31.118000   
...                  ...          ...                     ...        ...   
2017-07-28         179.0          2.0                     3.0  46.656000   
2017-07-29         196.0          0.0                     2.0  82.296000   
2017-07-30         380.0          0.0                     2.0  72.895996   
2017-07-31         247.0          1.0                     1.0  96.937996   
2017-08-01         219.0          0.0                     1.0  72.303000   

                                                                        
store_nbr                                                               
family     PREPARED FOODS   PRODUCE SCHOOL AND OFFICE SUPPLIES SEAFOOD  
date                                                                    
2013-01-01            0.0     0.000                        0.0     0.0  
2013-01-02           61.0     0.000                        0.0     3.0  
2013-01-03            1.0     0.000                        0.0     2.0  
2013-01-04           38.0     0.000                        0.0     2.0  
2013-01-05           32.0     0.000                        0.0     1.

In [16]:
fourier = CalendarFourier(freq='W', order=4)

dp = DeterministicProcess(index=y.index,
                          constant=False,
                          order=1,
                          seasonal=False,
                          additional_terms=[fourier],
                          drop=True)
X = dp.in_sample()

In [17]:
X

,trend,"sin(1,freq=W-SUN)","cos(1,freq=W-SUN)","sin(2,freq=W-SUN)","cos(2,freq=W-SUN)","sin(3,freq=W-SUN)","cos(3,freq=W-SUN)"
date,,,,,,,
2013-01-01,1.0,0.781831,0.623490,0.974928,-0.222521,0.433884,-0.900969
2013-01-02,2.0,0.974928,-0.222521,-0.433884,-0.900969,-0.781831,0.623490
2013-01-03,3.0,0.433884,-0.900969,-0.781831,0.623490,0.974928,-0.222521
2013-01-04,4.0,-0.433884,-0.900969,0.781831,0.623490,-0.974928,-0.222521
2013-01-05,5.0,-0.974928,-0.222521,0.433884,-0.900969,0.781831,0.623490
...,...,...,...,...,...,...,...
2017-07-28,1666.0,-0.433884,-0.900969,0.781831,0.623490,-0.974928,-0.222521
2017-07-29,1667.0,-0.974928,-0.222521,0.433884,-0.900969,0.781831,0.623490
2017-07-30,1668.0,-0.781831,0.623490,-0.974928,-0.222521,-0.433884,-0.900969


In [18]:
model = Ridge(fit_intercept=True, solver='auto', alpha=0.4,  random_state=23)
model.fit(X, y)
y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)

In [19]:
y_pred

sales                                                          \
store_nbr          1                                                            
family     AUTOMOTIVE BABY CARE    BEAUTY    BEVERAGES     BOOKS BREAD/BAKERY   
date                                                                            
2013-01-01   2.540360       0.0  1.728521   840.033778 -0.146453   279.082322   
2013-01-02   2.201488       0.0  1.760195  1036.435519 -0.123705   344.905988   
2013-01-03   1.729363       0.0  1.665176   767.077686 -0.186165   288.408627   
2013-01-04   2.487739       0.0  1.519761   929.264729 -0.152032   296.140143   
2013-01-05   2.378119       0.0  1.897685  1016.372408 -0.161345   283.182801   
...               ...       ...       ...          ...       ...          ...   
2017-07-28   5.136396       0.0  3.229990  2536.205787  0.429954   437.490385   
2017-07-29   5.026775       0.0  3.607914  2623.313466  0.420642   424.533043   
2017-07-30   2.728227       0.0  2.135452  1589.341818  0.354896   219.362473   
2017-07-31   4.621120       0.0  3.490820  2499.850277  0.424825   447.780744   
2017-08-01   5.200173       0.0  3.445953  2453.742939  0.437984   421.027902   

                                                            ...            \
store_nbr                                                   ...        54   
family     CELEBRATION    CLEANING       DAIRY        DELI  ... MAGAZINES   
date                                                        ...             
2013-01-01   -0.056561  657.836798  459.611900  110.219001  ... -0.380811   
2013-01-02    1.317640  804.283357  589.496246  120.866380  ... -0.364214   
2013-01-03    1.683820  645.837487  422.714087   99.011880  ... -0.383919   
2013-01-04    3.755064  689.962412  482.596195  131.915014  ... -0.235638   
2013-01-05   -1.021784  567.640965  514.319087  114.769561  ... -0.004281   
...                ...         ...         ...         ...  ...       ...   
2017-07-28   22.892469  766.531638  855.905376  167.532986  ...  1.169898   
2017-07-29   18.115621  644.210191  887.628268  150.387532  ...  1.401255   
2017-07-30   12.049751  285.929563  500.104846   75.171957  ...  1.292982   
2017-07-31   23.268028  720.412483  853.904963  155.481165  ...  1.095966   
2017-08-01   19.161447  734.728518  834.493382  145.986987  ...  1.030645   

                                                                          \
store_nbr                                                                  
family          MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                       
2013-01-01  41.710109    116.035302    -0.048619               -0.659626   
2013-01-02  32.717053     94.718718    -0.034999               -0.992438   
2013-01-03  45.860135     83.518203    -0.064139               -1.042451   
2013-01-04  38.894046    106.019992    -0.072282               -0.601104   
2013-01-05  49.303295    174.173614    -0.010008               -0.076840   
...               ...           ...          ...                     ...   
2017-07-28  52.971577    188.168876     0.207069                4.379622   
2017-07-29  63.380826    256.322499     0.269342                4.903886   
2017-07-30  76.641724    342.859313     0.278720                5.198941   
2017-07-31  58.190452    233.902512     0.252615                4.439688   
2017-08-01  55.846932    198.530181     0.231908                4.342078   

                                                                             \
store_nbr                                                                     
family        POULTRY PREPARED FOODS     PRODUCE SCHOOL AND OFFICE SUPPLIES   
date                                                                          
2013-01-01  38.083160      43.462535   35.742870                  -0.817239   
2013-01-02  29.588979      32.957768 -144.154291                  -0.647270   
2013-01-03  31.269227      35

In [20]:
y_pred   = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()

y_target['sales_pred'] = y_pred['sales'].clip(0.) # Sales should be >= 0
y_target

,date,store_nbr,family,sales,sales_pred
0,2013-01-01,1,AUTOMOTIVE,0.000,2.540360
1,2013-01-01,1,BABY CARE,0.000,0.000000
2,2013-01-01,1,BEAUTY,0.000,1.728521
3,2013-01-01,1,BEVERAGES,0.000,840.033778
4,2013-01-01,1,BOOKS,0.000,0.000000
...,...,...,...,...,...
2975935,2017-08-01,54,POULTRY,72.303,64.829030
2975936,2017-08-01,54,PREPARED FOODS,114.000,85.259246
2975937,2017-08-01,54,PRODUCE,1045.444,878.782019
2975938,2017-08-01,54,SCHOOL AND OFFICE SUPPLIES,0.000,1.741871


In [50]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

r2_score(y_target['sales'], y_target['sales_pred'])

0.8884022526169201

In [52]:
mean_absolute_percentage_error(y_target['sales'], y_target['sales_pred'])

4.173940510670743e+16

In [23]:
X_test = dp.out_of_sample(steps=16)
X_test

,trend,"sin(1,freq=W-SUN)","cos(1,freq=W-SUN)","sin(2,freq=W-SUN)","cos(2,freq=W-SUN)","sin(3,freq=W-SUN)","cos(3,freq=W-SUN)"
2017-08-02,1671.0,0.974928,-0.222521,-0.433884,-0.900969,-0.781831,0.623490
2017-08-03,1672.0,0.433884,-0.900969,-0.781831,0.623490,0.974928,-0.222521
2017-08-04,1673.0,-0.433884,-0.900969,0.781831,0.623490,-0.974928,-0.222521
2017-08-05,1674.0,-0.974928,-0.222521,0.433884,-0.900969,0.781831,0.623490
2017-08-06,1675.0,-0.781831,0.623490,-0.974928,-0.222521,-0.433884,-0.900969
2017-08-07,1676.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
2017-08-08,1677.0,0.781831,0.623490,0.974928,-0.222521,0.433884,-0.900969
2017-08-09,1678.0,0.974928,-0.222521,-0.433884,-0.900969,-0.781831,0.623490
2017-08-10,1679.0,0.433884,-0.900969,-0.781831,0.623490,0.974928,-0.222521
2017-08-11,1680.0,-0.433884,-0.900969,0.781831,0.623490,-0.974928,-0.222521


In [37]:
# Test actual pred predictions

stest = '2017-08-16'
etest = '2017-08-31'

X_test = dp.out_of_sample(steps=16)
X_test

,trend,"sin(1,freq=W-SUN)","cos(1,freq=W-SUN)","sin(2,freq=W-SUN)","cos(2,freq=W-SUN)","sin(3,freq=W-SUN)","cos(3,freq=W-SUN)"
2017-08-16,138.0,0.974928,-0.222521,-0.433884,-0.900969,-0.781831,0.623490
2017-08-17,139.0,0.433884,-0.900969,-0.781831,0.623490,0.974928,-0.222521
2017-08-18,140.0,-0.433884,-0.900969,0.781831,0.623490,-0.974928,-0.222521
2017-08-19,141.0,-0.974928,-0.222521,0.433884,-0.900969,0.781831,0.623490
2017-08-20,142.0,-0.781831,0.623490,-0.974928,-0.222521,-0.433884,-0.900969
2017-08-21,143.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
2017-08-22,144.0,0.781831,0.623490,0.974928,-0.222521,0.433884,-0.900969
2017-08-23,145.0,0.974928,-0.222521,-0.433884,-0.900969,-0.781831,0.623490
2017-08-24,146.0,0.433884,-0.900969,-0.781831,0.623490,0.974928,-0.222521
2017-08-25,147.0,-0.433884,-0.900969,0.781831,0.623490,-0.974928,-0.222521


In [31]:
X_test.index.name = 'date'

In [32]:

sales_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
sales_pred = sales_pred.stack(['store_nbr', 'family'])

In [35]:
sales_pred = sales_pred.reset_index()

In [40]:
sales_pred = sales_pred.set_index([ 'store_nbr', 'family', 'date'])

In [42]:
sales_pred = sales_pred.loc[sales_pred.index.isin(train.index)]
sales_pred

sales
store_nbr family                     date                   
1         AUTOMOTIVE                 2017-08-02     4.861300
          BABY CARE                  2017-08-02     0.000000
          BEAUTY                     2017-08-02     3.477627
          BEVERAGES                  2017-08-02  2650.144681
          BOOKS                      2017-08-02     0.460732
...                                                      ...
54        POULTRY                    2017-08-15    65.053381
          PREPARED FOODS             2017-08-15    85.609848
          PRODUCE                    2017-08-15   885.853647
          SCHOOL AND OFFICE SUPPLIES 2017-08-15     1.763338
          SEAFOOD                    2017-08-15     2.022288

[24948 rows x 1 columns]

In [43]:
sales_pred['sales_preds'] = train['sales']

<ipython-input-43-1e097d91203e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_pred['sales_preds'] = train['sales']


In [44]:
sales_pred

sales  sales_preds
store_nbr family                     date                                
1         AUTOMOTIVE                 2017-08-02     4.861300        4.000
          BABY CARE                  2017-08-02     0.000000        0.000
          BEAUTY                     2017-08-02     3.477627        2.000
          BEVERAGES                  2017-08-02  2650.144681     2645.000
          BOOKS                      2017-08-02     0.460732        0.000
...                                                      ...          ...
54        POULTRY                    2017-08-15    65.053381       59.619
          PREPARED FOODS             2017-08-15    85.609848       94.000
          PRODUCE                    2017-08-15   885.853647      915.371
          SCHOOL AND OFFICE SUPPLIES 2017-08-15     1.763338        0.000
          SEAFOOD                    2017-08-15     2.022288        3.000

[24948 rows x 2 columns]

In [45]:
r2_score(sales_pred['sales'], sales_pred['sales_preds'])

0.9161389952252464

## Test performance 91% r squared